# Prefix Sum
A Prefix Sum array allow us to **get range sum of in the array in O(1) time**

Therefore, it the question is asking about **Subarray Sum**, try using prefix sum!!!

---
### Q1: Sum Range Query (LC.303)
*Given an integer array nums, handle multiple queries of the following type:*   
- *Calculate the sum of the elements of nums between indices left and right inclusive where left <= right.*

*Implement the NumArray class:*         
- *`NumArray(int[] nums)`: Initializes the object with the integer array nums.*
- *`int sumRange(int left, int right)`: Returns the sum of the elements of nums between indices left and right inclusive (i.e. nums[left] + nums[left + 1] + ... + nums[right]).*

**Solution:**                
We define `prefixSum[i]: the sum of the first i elements`

In [13]:
class NumArray(object):
    def __init__(self, nums):
        self.prefixSum = [0] * (len(nums) + 1)
        for i in range(1, len(nums) + 1):
            self.prefixSum[i] = self.prefixSum[i - 1] + nums[i - 1]

    def sumRange(self, left, right):
        return self.prefixSum[right + 1] - self.prefixSum[left]

Note that We can also define `prefixSum[i]: the sum of the subarray that ends with nums[i]`      
Later in this note we might use either implementation

---
### Q2. Subarray Sum Equals K (LC.560)
*Given an array of integers nums and an integer k, return the total number of subarrays whose sum equals to k.*      
*A subarray is a contiguous non-empty sequence of elements within an array.*

**Solution:**            
We first construct and then iterate through the prefix sum array.    

The key point is again this: If `prefixSum[j] - prefixSum[i] == k`, then it means that the sum of `nums[i:j]` is k.    

Therefore we use a hashmap to keep track of the frequencies of each prefix sum.       

Then we calculate the number of subarray has the sum of `k` that ends in position `j` for each index `j`:
- When we are at position `j` and `prefixSum[j] - prefixSum[i] == k`, we simply add the frequencies of `prefixSum[i]` currently stored in the hashmap

In [18]:
class Solution(object):
    def subarraySum(self, nums, k):
        n = len(nums)
        prefixSum = [0] * (n + 1)
        for i in range(1, n + 1):
            prefixSum[i] = prefixSum[i - 1] + nums[i - 1]
        frequencies = defaultdict(int)
        ans = 0
        for i in range(n + 1):
            if prefixSum[i] - k in frequencies:
                ans += frequencies[prefixSum[i] - k]
            frequencies[prefixSum[i]] += 1
        return ans

**Optimization:**            
In fact, we don't even need a prefix sum array since we have stored the information we needed in a hashmap.           
Therefore, we can just use a single variable to represent the current prefix sum.           
Note that we should manually add `frequencies[0] = 1` since the prefixSum 0 already appears when there is no element

In [ ]:
class Solution(object):
    def subarraySum(self, nums, k):
        n = len(nums)
        prefixSum = 0
        frequencies = defaultdict(int)
        frequencies[0] = 1
        ans = 0
        for i in range(n):
            prefixSum += nums[i]
            if prefixSum - k in frequencies:
                ans += frequencies[prefixSum - k]
            frequencies[prefixSum] += 1
        return ans

---
### Q3. Maximum Length Of Subarray Sum Equals K
*Given an array of integers nums and an integer k, return the length of the longest subarray whose sum equals to k.*      

**Solution:**          
The question is pretty similar to the previous one, we just need to change the definition of our hashmap.          
Instead of counting frequency, our hashmap stores the leftmost position where a prefixSum appears.            
Again, `firstAppears[0] = -1` because the prefix sum is 0 when there is no element

In [46]:
class Solution(object):
    def subarrayLength(self, nums, k):
        map = {}
        map[0] = -1                  # Important: prefix sum 0 exists even when no elements are considered
        maxLen = 0
        sum = 0
        for i in range(n):
            sum += arr[i]
            if sum - aim in map:
                maxLen = max(maxLen, i - map[sum - aim])
            if sum not in map:       # Only update when we first see a specific prefix sum since we are looking for the leftmost index of that sum
                map[sum] = i
        return maxLen

---
### Q4: Maximum Length Of Subarray Equal Sign
*Given an array of integers nums and an integer k, return the length of the longest subarray within which the number of positive and negative numbers is the same*  

**Solution:**            
We just need one simple trick to transform the problem to the exact same problem as last one:   

**We treat positive number as 1 and negative numbers as -1**           

Then we are just looking for the longest subarray whose sum is 0

In [50]:
class Solution(object):
    def subarrayLength(self, nums):
        map = {}
        map[0] = -1 
        maxLen = 0
        sum = 0
        for i in range(n):
            sum += 1 if nums[i] > 0 else -1 if nums[i] < 0 else 0
            if sum in map:
                maxLen = max(maxLen, i - map[sum])
            if sum not in map:
                map[sum] = i
        return maxLen

---
### Q5: Longest Well Performing Interval (LC.1124) 
*We are given `hours`, a list of the number of hours worked per day for a given employee.*           
*A day is considered to be a tiring day if and only if the number of hours worked is (strictly) greater than 8.*           
*A well-performing interval is an interval of days for which the number of tiring days is strictly larger than the number of non-tiring days.*          
*Return the length of the longest well-performing interval.*

**Solution:**           
Again, treat a `hours[i] > 8` as `1` and a `hours[i] <= 8` as `-1`         
Then we are looking for a subarray sum that is positive
So given an index i, we check:
- If `hours[i] > 0`, then `ans = i + 1`, because the subarray `[0:i]` fulfill the requirement
- If `hours[i] < 0`, we look for a prefix sum that is smaller current prefix sum. In fact, we **only need to look for `presum - 1`!**
  - The reason that we only look for `presum - 1` is because our prefix sum increase or decrease only by step of 1
  - Suppose currently `presum == 3`, we know that `-4, -5, -6, ...` all works
  - But because of our prefix sum start at `0`, it must somehow first get to `-4` before it becomes `-5`
  - Therefore, there must be an index at which `presum == -4` and is **EARLIER** than `-5`
  - So we only look for `-4`

In [58]:
class Solution(object):
    def longestWPI(self, hours):
        presum = 0
        firstAppear = {}
        firstAppear[0] = -1
        maxLen = 0
        for i in range(len(hours)):
            presum += 1 if hours[i] > 8 else -1
            if presum > 0:
                maxLen = i + 1
            else:
                if presum - 1 in firstAppear:
                    maxLen = max(maxLen, i - firstAppear[presum - 1])
            if presum not in firstAppear:
                firstAppear[presum] = i
        return maxLen

---
### Q6. Make Sum Divisible By P (LC.1590)
*Given an array of positive integers nums, remove the smallest subarray (possibly empty) such that the sum of the remaining elements is divisible by p. It is not allowed to remove the whole array.*     
*Return the length of the smallest subarray that you need to remove, or -1 if it's impossible.*       
*A subarray is defined as a contiguous block of elements in the array.*

**Solution:**         
This problem is a little different. We still use prefix sum, but the thing we are looking for is different.       

We iterate through the array, and for each index `i`, we try to find:
- If we have to delete an subarray that ends in index `i`, what is the shortest length to make the remaining parts divisible by P?

First we find the remainder of the entire array, it that is 0, return 0 since we don't need to delete anything.

Otherwise we create a hashmap:
- `lastAppear[sumRemainder]`: the rightmost index `i` at which the `sum[0:i]` has a remainder of `sumRemainder` when divided by `P`

Now lets analyze what to look for. Lets suppose the remainder of the whole array is 7. We can break the array into 3 parts:
- Part A. the left part after we delete
- Part B. the part we want to delete
- Part C. the part we haven't visit

The parts look like this:        
|-----A-----|--B--|-----C----|        

Our `curmod` is the remainder of the prefix sum of A + B, and our `mod` is the remainder of the entire array (A + B + C)      
- if `curmod <= mod`
  - Then we look for **`curmod + P - mod`**
  - for eaxmple, if `curmod == 3` and `mod == 4`, this means that `Cmod == 1`. Therefore, we need `Amod == 6`
- if `curmod > mod`
  - Then we look for **`curmod - mod`**
  - for example, if `curmod == 5` and `mod == 3`, this means that `Cmod == 5`. Therefore we need `Amod == 2`

In [69]:
class Solution(object):
    def minSubarray(self, nums, p):
        mod = sum(nums) % p
        if mod == 0:
            return 0
        curmod = 0
        lastAppear = {}
        lastAppear[0] = -1
        ans = float('inf')
        for i in range(len(nums)):
            curmod = (curmod + nums[i]) % p
            find = curmod - mod if curmod >= mod else curmod + p - mod
            if find in lastAppear:
                ans = min(ans, i - lastAppear[find])
            lastAppear[curmod] = i
        return ans if ans != len(nums) else -1

---
### Q7. Find The Longest Substring Containing Vowels In Even Count
*Given the string s, return the size of the longest substring containing each vowel an even number of times. That is, 'a', 'e', 'i', 'o', and 'u' must appear an even number of times.*  

**Solution:**         
Because we need to keep track of the prefix cnt for all 5 letters, using a hashmap along is not enough.
- Therefore, use a **bitmask** to represent the status.

The bitmask has 5 bit, each represent whether a letter has appeared even or odd times.    
If a letter has a even count, it is 0 in the bitmask, otherwise 1.        
- For example, if the a is odd, b is even, c is odd, d is even, e is even, status would be: `10100`       

Also, because we only need to keep track of the state of 5 bits, we can simply use a int[32] instead of a hash map
- `map[status]`: Where is the index that status appears for the first time?

We need to find a subarray in which aeiou are all even. Therefore, given a current state, we need to **look for the same state on the left!**     
- This is because even - even = even, and odd - odd = even. therefore if the parity of all 5 characters are the same for index i and j, then [i,j] is a valid subarray.     

In [79]:
class Solution(object):
    def findTheLongestSubstring(self, s):
        n = len(s)
        map = [-2] * 32                 # -2 means that this state has never appeared before
        map[0] = -1                     # 0 = '00000', since there is nothing in a empty string, count for aeiou are all even
        ans = 0
        status = 0
        
        for i in range(n):
            m = self.move(s[i])         # Given the current letter, find out which bit we need to change
            if m != -1:                 # If current letter is a vowel, we need to update status
                status ^= 1 << m        # Flip the corresponding bit for the vowel
            
            if map[status] != -2:       # If this status has been seen before, calculate the length of the subarray
                ans = max(ans, i - map[status])
            else:
                map[status] = i         # Record the first time this status occurs
        
        return ans

    def move(self, char):
        if char == 'a': return 0
        if char == 'e': return 1
        if char == 'i': return 2
        if char == 'o': return 3
        if char == 'u': return 4
        return -1                        # For non-vowel characters